In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"..\data\raw\train.csv")

In [3]:
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  str    
 3   Manufacturer      19237 non-null  str    
 4   Model             19237 non-null  str    
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  str    
 7   Leather interior  19237 non-null  str    
 8   Fuel type         19237 non-null  str    
 9   Engine volume     19237 non-null  str    
 10  Mileage           19237 non-null  str    
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  str    
 13  Drive wheels      19237 non-null  str    
 14  Doors             19237 non-null  str    
 15  Wheel             19237 non-null  str    
 16  Color             19237 non-null  str    
 17  Airb

In [5]:
X = df.drop(['ID','Price','Prod. year','Wheel','Doors'],axis=1)
y = df.iloc[:,1:2]

In [6]:
X['Mileage'] = X['Mileage'].str.split().str[0].astype(int)

In [7]:
X['Levy'] = X['Levy'].replace('-',0)
X['Levy'] = X['Levy'].astype(int)

In [8]:
X['is_turbo'] = X['Engine volume'].str.contains('Turbo', case=False, na=False).astype(int)

In [9]:
X['Engine volume'] = (
    X['Engine volume']
    .str.extract(r'(\d+\.?\d*)')
    .astype(float)
)

In [10]:
X

,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Color,Airbags,is_turbo
0,1399,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,Silver,12,0
1,1018,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000,6.0,Tiptronic,4x4,Black,8,0
2,0,HONDA,FIT,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,Black,2,0
3,862,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,White,0,0
4,446,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,Silver,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,0,MERCEDES-BENZ,CLK 200,Coupe,Yes,CNG,2.0,300000,4.0,Manual,Rear,Silver,5,1
19233,831,HYUNDAI,Sonata,Sedan,Yes,Petrol,2.4,161600,4.0,Tiptronic,Front,Red,8,0
19234,836,HYUNDAI,Tucson,Jeep,Yes,Diesel,2.0,116365,4.0,Automatic,Front,Grey,4,0
19235,1288,CHEVROLET,Captiva,Jeep,Yes,Diesel,2.0,51258,4.0,Automatic,Front,Black,4,0


In [11]:
cat_col= X.select_dtypes(include='object')

C:\Users\anmolp\AppData\Local\Temp\ipykernel_10236\781652912.py:1: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_col= X.select_dtypes(include='object')


In [12]:
OHE_cols = []
MED_cols = []
FED_cols = []

for col in cat_col:
    nunique = X[col].nunique()
    
    if nunique <= 5:
        OHE_cols.append(col)
    elif 6 <= nunique <= 15:
        MED_cols.append(col)
    else:
        FED_cols.append(col)

In [13]:
print("One-Hot Encoding Columns:", OHE_cols)
print("Mean/Target Encoding Columns:", MED_cols)
print("Frequency Encoding Columns:", FED_cols)

One-Hot Encoding Columns: ['Leather interior', 'Gear box type', 'Drive wheels']
Mean/Target Encoding Columns: ['Category', 'Fuel type']
Frequency Encoding Columns: ['Manufacturer', 'Model', 'Color']


### One-Hot Encoding

In [14]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

ohe_array = enc.fit_transform(X[OHE_cols]).toarray()

ohe_cols = enc.get_feature_names_out(OHE_cols)

ohe_df = pd.DataFrame(ohe_array, columns=ohe_cols, index=X.index)

X = pd.concat([X.drop(columns=OHE_cols), ohe_df], axis=1)

### Target Encoding

In [15]:
for col in MED_cols:
    mean_map = df.groupby(col)['Price'].mean()
    X[col] = X[col].map(mean_map)

### Frequency Encoding

In [16]:
for col in FED_cols:
    freq = X[col].value_counts()
    X[col] = X[col].map(freq)

In [17]:
X

,Levy,Manufacturer,Model,Category,Fuel type,Engine volume,Mileage,Cylinders,Color,Airbags,is_turbo,Leather interior_No,Leather interior_Yes,Gear box type_Automatic,Gear box type_Manual,Gear box type_Tiptronic,Gear box type_Variator,Drive wheels_4x4,Drive wheels_Front,Drive wheels_Rear
0,1399,982,173,23684.285949,10849.320570,3.5,186005,6.0,3792,12,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1018,1069,23,23684.285949,17377.351527,3.0,192000,6.0,5033,8,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0,977,446,11381.245873,17377.351527,1.3,200000,4.0,5033,2,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,862,1111,149,23684.285949,10849.320570,2.5,168966,4.0,4489,0,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,446,977,446,11381.245873,17377.351527,1.3,91901,4.0,3792,4,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,0,2076,8,21250.406015,8509.591093,2.0,300000,4.0,3792,5,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
19233,831,3769,1079,14231.545902,17377.351527,2.4,161600,4.0,639,8,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
19234,836,3769,429,23684.285949,30631.071358,2.0,116365,4.0,2375,4,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
19235,1288,1069,140,23684.285949,30631.071358,2.0,51258,4.0,5033,4,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.33,random_state=42)

st = StandardScaler()

X_train = st.fit_transform(X_train)

X_test = st.transform(X_test)

In [35]:
X_test

array([[-0.66264801, -0.67053538, -0.51158406, ..., -0.51213161,
         0.69835254, -0.36934057],
       [-1.12958883,  0.03643864, -0.70279366, ..., -0.51213161,
        -1.43194152,  2.70752815],
       [-1.12958883,  0.03643864, -0.7399733 , ..., -0.51213161,
        -1.43194152,  2.70752815],
       ...,
       [ 0.02424177, -1.2420119 , -0.72669486, ...,  1.95262309,
        -1.43194152, -0.36934057],
       [ 0.81569745, -1.21814187, -0.87806912, ..., -0.51213161,
         0.69835254, -0.36934057],
       [-0.07311269, -0.67053538, -0.09198523, ..., -0.51213161,
         0.69835254, -0.36934057]], shape=(12889, 20))

### Model Traning

In [19]:
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor

In [20]:
models = {

    "LinearRegression": {
        "model": LinearRegression(),
        "params": {
            "fit_intercept": [True, False]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [0.1, 1.0, 10.0]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=5000),
        "params": {
            "alpha": [0.01, 0.1, 1.0]
        }
    },

    "ElasticNet": {
        "model": ElasticNet(max_iter=5000),
        "params": {
            "alpha": [0.01, 0.1, 1.0],
            "l1_ratio": [0.2, 0.5, 0.8]
        }
    },

    "DecisionTree": {
        "model": DecisionTreeRegressor(random_state=42),
        "params": {
            "max_depth": [5, 10, None],
            "min_samples_split": [2, 5, 10]
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [10, None],
            "min_samples_split": [2, 5]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1],
            "max_depth": [3, 5]
        }
    },

    "SVR": {
        "model": SVR(),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"],
            "gamma": ["scale", "auto"]
        }
    },

    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [3, 5, 7],
            "weights": ["uniform", "distance"]
        }
    },

    "SGDRegressor": {
        "model": SGDRegressor(max_iter=1000, tol=1e-3),
        "params": {
            "alpha": [0.0001, 0.001],
            "penalty": ["l2", "l1"]
        }
    }
}

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

for name, mp in models.items():
    grid = GridSearchCV(
        mp["model"],
        mp["params"],
        cv=5,
        scoring="r2",
        n_jobs=-1
    )
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    print(name)
    print("Accuracy:", grid.score(X_test, y_test))
    print("MSE", mean_squared_error(y_test, y_pred))
    print("R2 Score:", r2_score(y_test, y_pred))
    print("Best Score:", grid.best_score_)
    print("Best Params:", grid.best_params_)
    print("-" * 40)

LinearRegression
Accuracy: 0.0008780162220058951
MSE 54004976208.22574
R2 Score: 0.0008780162220058951
Best Score: 0.18359896768686995
Best Params: {'fit_intercept': True}
----------------------------------------
Ridge
Accuracy: 0.0008791824968762407
MSE 54004913168.22897
R2 Score: 0.0008791824968762407
Best Score: 0.18361178971116784
Best Params: {'alpha': 10.0}
----------------------------------------
Lasso
Accuracy: 0.0008780135449478754
MSE 54004976352.927246
R2 Score: 0.0008780135449478754
Best Score: 0.18359895553934769
Best Params: {'alpha': 0.01}
----------------------------------------
ElasticNet
Accuracy: 0.000883633791908478
MSE 54004672564.89303
R2 Score: 0.000883633791908478
Best Score: 0.1836304640190798
Best Params: {'alpha': 0.01, 'l1_ratio': 0.2}
----------------------------------------
DecisionTree
Accuracy: 0.002107141994638906
MSE 53938538967.14236
R2 Score: 0.002107141994638906
Best Score: 0.31765522389751466
Best Params: {'max_depth': None, 'min_samples_split': 10

d:\ML_REG\.venv\Lib\site-packages\sklearn\base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest
Accuracy: 0.0035063227493988824
MSE 53862909840.170204
R2 Score: 0.0035063227493988824
Best Score: 0.5888037798628103
Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
----------------------------------------


d:\ML_REG\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GradientBoosting
Accuracy: 0.0035409216860541326
MSE 53861039683.39131
R2 Score: 0.0035409216860541326
Best Score: 0.5665294096023521
Best Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
----------------------------------------


d:\ML_REG\.venv\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR
Accuracy: 0.00011794932971675998
MSE 54046059674.61041
R2 Score: 0.00011794932971675998
Best Score: 0.10328748982217115
Best Params: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
----------------------------------------
KNN
Accuracy: 0.002660814392046351
MSE 53908611625.80204
R2 Score: 0.002660814392046351
Best Score: 0.4497971380174962
Best Params: {'n_neighbors': 7, 'weights': 'distance'}
----------------------------------------
SGDRegressor
Accuracy: 0.0007208171372610206
MSE 54013473201.55532
R2 Score: 0.0007208171372610206
Best Score: 0.18450309493519484
Best Params: {'alpha': 0.001, 'penalty': 'l2'}
----------------------------------------


d:\ML_REG\.venv\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
import numpy as np

results = {}

for name, mp in models.items():
    print(f"Training {name}...")

    random_search = RandomizedSearchCV(
        estimator=mp["model"],
        param_distributions=mp["params"],
        n_iter=20,               # number of random combinations
        cv=5,
        scoring="r2",
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(X_train, y_train)

    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)

    results[name] = {
        "Best CV Score": random_search.best_score_,
        "Test R2": r2_score(y_test, y_pred),
        "Best Params": random_search.best_params_
    }

Training LinearRegression...
Training Ridge...


d:\ML_REG\.venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 2 is smaller than n_iter=20. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\ML_REG\.venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 3 is smaller than n_iter=20. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Training Lasso...


d:\ML_REG\.venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 3 is smaller than n_iter=20. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Training ElasticNet...


...

d:\ML_REG\.venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\ML_REG\.venv\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training KNN...


d:\ML_REG\.venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Training SGDRegressor...


d:\ML_REG\.venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\ML_REG\.venv\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
print(results[name])

{'Best CV Score': np.float64(0.18343381795994376), 'Test R2': 0.0006828197662835933, 'Best Params': {'penalty': 'l2', 'alpha': 0.001}}


RandomForest
Accuracy: 0.0035063227493988824
MSE 53862909840.170204
R2 Score: 0.0035063227493988824
Best Score: 0.5888037798628103
Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}

In [ ]:
regr = RandomForestRegressor(**results['RandomForest']['Best Params'])

In [30]:
regr.fit(X_train, y_train)

d:\ML_REG\.venv\Lib\site-packages\sklearn\base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [31]:
regr.score(X_test, y_test)

0.003490196413455271

In [ ]:
final_model = GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error

print("R2:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))


d:\ML_REG\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


R2: 0.002502874474830552
RMSE: 53917148662.930695
